In [1]:
import numpy as np
import webdataset as wds
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms 
import os
import random

PATH_TO_DATA = "/mnt/analysis/analysis/rand_sharded_data/" 

In [2]:
image_normalize = transforms.Normalize(
                  mean=[0.17960437768666657, 0.14584139607643212, 0.10744440357398845, 0.2583671063835548],
                  std=[0.059635202669355195, 0.04059554002618016, 0.03371736326989986, 0.06295501902505744]
)

forcing_normalize = transforms.Normalize(
                  mean=[444.9605606256559, 991.7980623653417, 0.00039606951184754176, 96111.04161525163, 0.006652783216819315, 314.3219695851273, 2.82168247768119],
                  std=[5.5216369223813535, 12.951212256256913, 0.0002824274832735609, 975.3770569179914, 0.00012386107613000674, 0.6004463118907452, 0.34279194598853185]
)

forcing_mean = torch.from_numpy(np.array([444.9605606256559, 991.7980623653417, 0.00039606951184754176, 96111.04161525163, 0.006652783216819315, 314.3219695851273, 2.82168247768119]))
forcing_std = torch.from_numpy(np.array([5.5216369223813535, 12.951212256256913, 0.0002824274832735609, 975.3770569179914, 0.00012386107613000674, 0.6004463118907452, 0.34279194598853185]))

lst_mean = torch.from_numpy(np.array([312.8291360088677]))
lst_std = torch.from_numpy(np.array([11.376636496297289]))

In [3]:
def create_train_test(path_to_data, train_perc, test_perc):
    files = []
    for dirpath, dirnames, filenames in os.walk(path_to_data):
        files.extend(filenames)
    
    saturated = files[:-1]
    unsaturated = files[-1]
    
    dataset = wds.WebDataset(path_to_data + "/" + unsaturated)
    counter = 0
    for data in dataset:
        counter += 1
    
    total_files = counter + len(saturated) * 10000
    training_data = total_files * train_perc //10000
    test_data_files = total_files * test_perc //10000

    training_data = random.sample(files, int(training_data))
    test_data = [file for file in files if file not in training_data]
    test_data = random.sample(test_data, int(test_data_files))
    # Get sample sizes of train and test data
    training_samples = 0
    testing_samples = 0
    
    for path in training_data:
        if path in saturated:
            training_samples += 10000
        elif path in unsaturated:
            training_samples += counter
            
    for path in test_data:
        if path in saturated:
            testing_samples += 10000
        elif path in unsaturated:
            testing_samples += counter
            
            
    # Convert to filename lists 
    training_filepath = []
    for dat in training_data:
        training_filepath.append(dat[6:12])
    training_path = path_to_data + "shard-" + "{" + ",".join(training_filepath) + "}" + ".tar"
    
    testing_filepath = []
    for dat in test_data:
        testing_filepath.append(dat[6:12])
    testing_path = path_to_data + "shard-{" + ",".join(testing_filepath) +"}.tar"
    train_data = wds.WebDataset(training_path).shuffle(10000, initial=10000).decode("rgb").rename(image="image.pyd", forcing="forcing.pyd", lst = "lst.pyd").to_tuple("image", "forcing", "lst")
    test_data = wds.WebDataset(testing_path).decode("rgb").shuffle(10000, initial=10000).rename(image="image.pyd", forcing="forcing.pyd", lst = "lst.pyd").to_tuple("image", "forcing", "lst")
            
    return (train_data, training_samples), (test_data, testing_samples)
    
(train_data, training_samples_len), (test_data, testing_samples_len) = create_train_test(PATH_TO_DATA, 0.85, 0.15)


In [4]:
class LSTModel(nn.Module):
    def __init__(self, input_shape=(4,33,33), forcing_shape=(1,7)):
        super(LSTModel, self).__init__()
        self.in_channels = input_shape[0]
        self.input_shape = input_shape
        
        self.conv1 = nn.Conv2d(in_channels=self.in_channels, out_channels=8, kernel_size=(3,3))
        self.conv1_bn = nn.BatchNorm2d(8)
        self.mp1 = nn.AvgPool2d(kernel_size=(4,4), stride=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(1,1))
        self.conv2_bn = nn.BatchNorm2d(8)
        self.mp2 = nn.MaxPool2d(kernel_size=(3, 3), stride=2)
#         self.conv3 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(1, 1))
#         self.conv3_bn = nn.BatchNorm2d(8)

        self.flatten_shape = None
        zero_ex = torch.zeros(input_shape).unsqueeze(0)
        zero_forcing = torch.zeros(forcing_shape)
        
        with torch.no_grad():
            self.convolutions(zero_ex, zero_forcing)
        
        print(self.flatten_shape)
        self.fc1 = nn.Linear(in_features=self.flatten_shape, out_features=128)
        self.fc_bn1 = nn.BatchNorm1d(128)
        self.drop1 = nn.Dropout(0)
        self.fc2 = nn.Linear(in_features=128, out_features=64)
        self.fc_bn2 = nn.BatchNorm1d(64)
        self.drop2 = nn.Dropout(0)
        self.fc3 = nn.Linear(in_features=64, out_features=1)
#         self.drop3 = nn.Dropout(0)
#         self.fc4 = nn.Linear(in_features=64, out_features=1)
        
    def convolutions(self, x, forcing):
        x = F.leaky_relu(self.conv1_bn(self.conv1(x)))
        x = self.mp1(x)
        x = F.leaky_relu(self.conv2_bn(self.conv2(x)))
        x = self.mp2(x)
#         x = F.leaky_relu(self.conv3_bn(self.conv3(x)))
#         x = self.mp3(x)

        # Reshape for linear
        x = x.view(x.shape[0], -1)
        x = torch.cat((x, forcing), dim=1)

        if self.flatten_shape is None:
            self.flatten_shape = x.shape[1]

        return x
    
    def forward(self, x, forcing):
        x = self.convolutions(x, forcing)
        x = F.leaky_relu(self.fc_bn1(self.fc1(x)))
        x = self.drop1(x)
        x = F.leaky_relu(self.fc_bn2(self.fc2(x)))
        x = self.drop2(x)
        x = self.fc3(x)
#         x = self.drop3(x)
#         x = self.fc4(x)
        
        return x
        

In [5]:
from tqdm import tqdm

EPOCHS = 100
LEARNING_RATE = 0.0005
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1024

train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, num_workers=6)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=6)

model = LSTModel().to(DEVICE)
model = torch.nn.DataParallel(model, device_ids=[0,1])
loss_fn = nn.SmoothL1Loss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.95)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
scheduler2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min")
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.0005)
test_loss = []
train_loss = []

lst_mean = lst_mean.to(DEVICE)
lst_std = lst_std.to(DEVICE)
forcing_mean = forcing_mean.to(DEVICE)
forcing_std = forcing_std.to(DEVICE)

def process_data(image, forcing, lst):
    image, forcing, lst = image.to(torch.float32).to(DEVICE), forcing.to(DEVICE), lst.to(DEVICE)
    # Image Transformations
    image = torch.clip(image, min=0, max=1)
#     image = image_normalize(image)
    # Forcing Transformation
    forcing = torch.div(torch.sub(forcing, forcing_mean), forcing_std).to(torch.float32)
    # LST Transformation
#     lst = torch.div(torch.sub(lst, lst_mean), lst_std).to(torch.float32).view(-1, 1)
    lst = lst.view(-1, 1).to(torch.float32)
    return image, forcing, lst

min_test_loss = np.inf    
for epoch in range(EPOCHS):
    print("****** EPOCH: [{}/{}] LR: {} ******".format(epoch, EPOCHS, round(optimizer.param_groups[0]['lr'], 6)))
    running_train_loss = 0
    train_n_iter = 0
    running_test_loss = 0
    test_n_iter = 0
    
    loop_train = tqdm(train_loader, total=(training_samples_len//BATCH_SIZE) + 1, leave=True)
    for idx, (image, forcing, lst) in enumerate(loop_train):
        image, forcing, lst = process_data(image, forcing, lst)
        optimizer.zero_grad()
        forward_out = model.forward(image, forcing)
        loss = loss_fn(forward_out, lst)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()
        train_n_iter += 1
        loop_train.set_postfix(train_loss=loss.item())
        
    loop_test = tqdm(test_loader, total=(testing_samples_len//BATCH_SIZE) + 1, leave=False)
    
    with torch.no_grad():
        for idx, (image, forcing, lst) in enumerate(loop_test):
            image, forcing, lst = process_data(image, forcing, lst)
            pred = model.forward(image, forcing)
            testloss = loss_fn(pred, lst)
            running_test_loss += testloss.item()
            test_n_iter += 1
            loop_test.set_postfix(test_loss=testloss.item())

    avg_train_loss = running_train_loss/train_n_iter
    train_loss.append(avg_train_loss)
    avg_test_loss = running_test_loss/test_n_iter
    test_loss.append(avg_test_loss)
    
    scheduler.step()
    scheduler2.step(avg_test_loss)
    if avg_test_loss < min_test_loss:
        print("Saving Model")
        min_test_loss = avg_test_loss
        torch.save(model.state_dict(), "lstmodel.pt")
    print("------ Train Loss: {}, Test Loss: {} ------".format(avg_train_loss, avg_test_loss))
            
        
        
        
        

295
****** EPOCH: [0/100] LR: 0.0005 ******


1713it [03:25,  8.33it/s, train_loss=6.77]                                                                                                            


Saving Model
------ Train Loss: 57.87471455213833, Test Loss: 6.839189325508318 ------
****** EPOCH: [1/100] LR: 0.000495 ******


1713it [03:36,  7.90it/s, train_loss=6.36]                                                                                                            


Saving Model
------ Train Loss: 6.068590689457027, Test Loss: 6.281690649296108 ------
****** EPOCH: [2/100] LR: 0.00049 ******


1713it [03:36,  7.89it/s, train_loss=6.1]                                                                                                             


Saving Model
------ Train Loss: 5.7069920970206365, Test Loss: 6.015624989020197 ------
****** EPOCH: [3/100] LR: 0.000485 ******


  5%|████▌                                                                                         | 82/1710 [00:17<05:39,  4.79it/s, train_loss=5.73]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
# plot lines
plt.plot(list(range(0,36)), train_loss, label = "train_loss")
plt.plot(list(range(0,36)), test_loss, label = "test_loss")
plt.legend()
plt.title("Training Error LST Prediction")
plt.savefig("Training Curve.png")
plt.ylim([0, 10])
plt.show()

In [ ]:
plt.savefig()